# **TP2 - Exercicio 1 - Grupo 11**

Bruno Miguel Fernandes Araújo - A97509

Tiago Emanuel Lemos Teixeira - A97666

# **Exercicio 1 - Algoritmo estendido de Euclides usando FOTS**


1. O algoritmo estendido de Euclides (EXA) aceita dois inteiros constantes  $\,a,b>0\,$  e devolve inteiros $r,s,t\,$ tais que  $\,a*s + b*t = r\,$  e  $\,r = \gcd(a,b)\,$. 
    Para além das variáveis $\,r,s,t\,$ o código requer 3 variáveis adicionais $\,r',s',t'\,$ que representam os valores de $\,r,s,t\,$ no “próximo estado”.

```Python
INPUT  a, b
assume  a > 0 and b > 0
r, r', s, s', t, t' = a, b, 1, 0, 0, 1
while r' != 0
  q = r div r'
  r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t' 
OUTPUT r, s, t


1. Construa um FOTS usando BitVector de tamanho $n$ que descreva o comportamento deste programa: identifique as variáveis do modelo, o estado inicial e a relação de transição.
2. Considere estado de erro quando $\,r=0\,$ ou alguma das variáveis atinge o “overflow”.
        Prove que o programa nunca atinge o estado de erro.
3. Prove que a relação de Bézout $\,$$ $\,a*s + b*t = r\, é um invariante do algoritmo.

# **Construção do FOTS**

Começaremos então por construir as funçãos que se encontram envolvidas neste processo.

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT
from pysmt.typing import BVType

Primeiramente a função **declare** onde declararemos todos os estados com variáveis do tipo $BVType(n)$ de forma a termos um bitvector de tamanho n associado a estas.
As variáveis presentes nestes são aquelas envolvidas no programa indicado no enunciado , ou seja, a , b , r ,r' , s , s',t,t' e q.
Além destas temos a variavél pc que é referente ao "program counter" e terá este comportamento no contexto do programa.

```Python
INPUT  a, b
pc = 0    assume  a > 0 and b > 0
pc = 1    r, r', s, s', t, t' = a, b, 1, 0, 0, 1
pc = 2    while r' != 0
pc = 3      q = r div r'
pc = 4      r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t' 
OUTPUT r, s, t



In [2]:
def declare(i):
    state = {}
    state['pc'] = Symbol('pc'+str(i),BVType(n))
    state['a'] = Symbol('a'+str(i),BVType(n))
    state['b'] = Symbol('b'+str(i),BVType(n))
    state['r'] = Symbol('r'+str(i),BVType(n))
    state['r_'] = Symbol('r_'+str(i),BVType(n))
    state['s'] = Symbol('s'+str(i),BVType(n))
    state['s_'] = Symbol('s_'+str(i),BVType(n))
    state['t'] = Symbol('t'+str(i),BVType(n))
    state['t_'] = Symbol('t_'+str(i),BVType(n))
    state['q'] = Symbol('q'+str(i),BVType(n))
    return state

De seguida a função **init** que trata das restrições no estado inicial, neste caso temos então.
$$ (\mathit{pc} = 0 \wedge (a \gt 0 \wedge a = a) \wedge (b \gt 0 \wedge b = b)) $$

Este a e b que estão a ser atribuidos aos state['a'] e state['b'] respetivamente , são os números dados como input.

In [3]:
def init(state):
    #pc == 0 && (a > 0 && a==a) && (b > 0 && b==b)
    PC = Equals(state['pc'],BV(0,n)) 
    A = And(BVUGT(state['a'],BV(0,n)),Equals(state['a'], BV(a,n)))
    B = And(BVUGT(state['b'],BV(0,n)),Equals(state['b'], BV(b,n)))
    return And(PC,A,B)


Por fim para as transições entre estados temos a função **trans** definida tendo em conta os seguintes predicados.

Notas: **next** como prefixo de uma variável refere a esta no proximo estado.

Transição 01:
$$ (\mathit{pc} = 0 \wedge \mathit{next pc} = 1 \wedge next a = a \wedge next b = b \wedge next r = a \wedge next r' = b \wedge next s = 1 \wedge next s' = 0 \wedge next t = 0 \wedge next t' = 1 ) $$

Transição 12:
$$(\mathit{pc} = 1 \wedge !( r' = 0 ) \wedge r = next r \wedge r' = next r' \wedge s = next s \wedge s' = next s' \wedge t = next t \wedge t' = next t' \wedge \mathit{next pc} = 2 )$$

Transição 23:
$$(\mathit{pc} = 2 \wedge r = next r \wedge r' = next r' \wedge s = next s \wedge s' = next s' \wedge t = next t \wedge t' = next t' \wedge \mathit{nextpc} =3 \wedge (nextq = r \div r' ) )$$

Transição 31:
$$(\mathit{pc} = 3 \wedge \mathit{nextpc} = 1 \wedge nextr = r' \wedge (nextr' = r - (q \times r')) \wedge nexts = s' \wedge      (nexts' = s - (q \times s')) \wedge nextt= t' \wedge (nextt' = t - (q \times t')) )$$

Transição 14:
$$(\mathit{pc} = 1 \wedge ( r' = 0 ) \wedge r = next r \wedge r' = next r' \wedge s = next s \wedge s' = next s' \wedge t = next t \wedge t' = next t' \wedge \mathit{next pc} = 4 )$$

Transição 44:
$$(\mathit{pc} = 4 \wedge r = next r \wedge r' = next r' \wedge s = next s \wedge s' = next s' \wedge t = next t \wedge t' = next t' \wedge \mathit{next pc} = 4 )$$


A transição 44 é para este permanecer caso k seja superior aos numero de passos necessários para determinar gcd(a,b).

In [4]:
def trans(curr, prox):
    
    t01= And(
             Equals(curr['pc'],BV(0,n)),#pc==0
             Equals(prox['pc'],BV(1,n)),#pc==1 
             Equals(prox['a'],curr['a']),#a==a
             Equals(prox['b'],curr['b']),#b==b
             Equals(prox['r'],curr['a']),#r==a
             Equals(prox['r_'],curr['b']),#r'==b
             Equals(prox['s'],BV(1,n)),#s==1
             Equals(prox['s_'],BV(0,n)),#s'==0
             Equals(prox['t'],BV(0,n)),#t==0
             Equals(prox['t_'],BV(1,n))#t'==1
            )
    
  #  print(t01)
  #  print("\n")
    
    t12= And(Equals(curr['pc'],BV(1,n)),#pc==1
             Not(Equals(curr['r_'],BV(0,n))),#r'!=0
             Equals(curr['r'],prox['r']),
             Equals(curr['r_'],prox['r_']),
             Equals(curr['s'],prox['s']),
             Equals(curr['s_'],prox['s_']),
             Equals(curr['t'],prox['t']),
             Equals(curr['t_'],prox['t_']),
             Equals(prox['pc'],BV(2,n))#pc==2
            )
    
  #  print(t12)
  #  print("\n")

    t23= And(Equals(curr['pc'],BV(2,n)),#pc==2
             Equals(curr['r'],prox['r']),
             Equals(curr['r_'],prox['r_']),
             Equals(curr['s'],prox['s']),
             Equals(curr['s_'],prox['s_']),
             Equals(curr['t'],prox['t']),
             Equals(curr['t_'],prox['t_']),
             Equals(prox['pc'],BV(3,n)),#pc==3
             Equals(prox['q'],BVUDiv(curr['r'],curr['r_']))
    )
        
  #  print(t23)
  #  print("\n")
    
    t31= And(Equals(curr['pc'],BV(3,n)),#pc==3
             Equals(prox['pc'],BV(1,n)),#pc==1
             Equals(prox['r'],curr['r_']),
             Equals(prox['r_'],BVSub(curr['r'],BVMul(curr['q'],curr['r_']))),
             Equals(prox['s'],curr['s_']),
             Equals(prox['s_'],BVSub(curr['s'],BVMul(curr['q'],curr['s_']))),
             Equals(prox['t'],curr['t_']),
             Equals(prox['t_'],BVSub(curr['t'],BVMul(curr['q'],curr['t_']))),
        )
    

  #  print(t31)
  #  print("\n")
             
    t14= And(Equals(curr['pc'],BV(1,n)),#pc==1
             Equals(curr['r_'],BV(0,n)),#r'==0
             Equals(curr['r'],prox['r']),
             Equals(curr['r_'],prox['r_']),
             Equals(curr['s'],prox['s']),
             Equals(curr['s_'],prox['s_']),
             Equals(curr['t'],prox['t']),
             Equals(curr['t_'],prox['t_']),
             Equals(prox['pc'],BV(4,n))#pc==4
            )
    
  #  print(t14)
  #  print("\n")
    
    t44= And(Equals(curr['pc'],BV(4,n)),
             Equals(curr['r'],prox['r']),
             Equals(curr['r_'],prox['r_']),
             Equals(curr['s'],prox['s']),
             Equals(curr['s_'],prox['s_']),
             Equals(curr['t'],prox['t']),
             Equals(curr['t_'],prox['t_']),
             Equals(curr['q'],prox['q']),
             Equals(prox['pc'],BV(4,n)),
            )
  #  print(t44)
  #  print("\n")
    
    return Or(t01,t12,t14,t23,t31,t44)

Por  fim temos a função **gera_traco** que depois de declarados os estados, o primeiro estado inicializado e as transições entre os estados respeitadas é então criado um solver que se for capaz de dar solve imprime o resultado se não devolve 
\>Not Feasible.


In [5]:
def gera_traco(k):
    
    states = [declare(i) for i in range(k)] # declarar estados
    with Solver("msat") as solver:
        solver.add_assertion(init(states[0])) # satisfazer a inicialização do primeiro estado
        
        for j in range(k-1): #satisfazer os restantes estados tendo em conta as transiçoes
            solver.add_assertion(trans(states[j],states[j+1]))
        
        if solver.solve(): 
            print(f"Input:\na = {a} , b = {b} \n\nExecução:")
            for i,s in enumerate(states):
                print(f"> state {i}: r = {solver.get_value(s['r']).bv_signed_value()}, r' = {solver.get_value(s['r_']).bv_signed_value()}, s = {solver.get_value(s['s']).bv_signed_value()}, s' = {solver.get_value(s['s_']).bv_signed_value()}, t = {solver.get_value(s['t']).bv_signed_value()}, t' = {solver.get_value(s['t_']).bv_signed_value()}, q = {solver.get_value(s['q']).bv_signed_value()}, pc = {solver.get_value(s['pc']).bv_signed_value()}.")
            print(f"\nOutput:\nr = {solver.get_value(s['r']).bv_signed_value()}, s = {solver.get_value(s['s']).bv_signed_value()}, t = {solver.get_value(s['t']).bv_signed_value()}")
        else:
            print(">Not Feasible.")

# **Exemplos:**


**Exemplo onde foi possível determinar gcd(a,b) em menos de k passos.**

In [6]:
#nº de Passos:
k=20

a=5
b=3

#tamanho do bitvector:
n=10 

gera_traco(k)

Input:
a = 5 , b = 3 

Execução:
> state 0: r = 13, r' = 12, s = 11, s' = 10, t = 9, t' = 8, q = 7, pc = 0.
> state 1: r = 5, r' = 3, s = 1, s' = 0, t = 0, t' = 1, q = 6, pc = 1.
> state 2: r = 5, r' = 3, s = 1, s' = 0, t = 0, t' = 1, q = 14, pc = 2.
> state 3: r = 5, r' = 3, s = 1, s' = 0, t = 0, t' = 1, q = 1, pc = 3.
> state 4: r = 3, r' = 2, s = 0, s' = 1, t = 1, t' = -1, q = 19, pc = 1.
> state 5: r = 3, r' = 2, s = 0, s' = 1, t = 1, t' = -1, q = 22, pc = 2.
> state 6: r = 3, r' = 2, s = 0, s' = 1, t = 1, t' = -1, q = 1, pc = 3.
> state 7: r = 2, r' = 1, s = 1, s' = -1, t = -1, t' = 2, q = 27, pc = 1.
> state 8: r = 2, r' = 1, s = 1, s' = -1, t = -1, t' = 2, q = 30, pc = 2.
> state 9: r = 2, r' = 1, s = 1, s' = -1, t = -1, t' = 2, q = 2, pc = 3.
> state 10: r = 1, r' = 0, s = -1, s' = 3, t = 2, t' = -5, q = 35, pc = 1.
> state 11: r = 1, r' = 0, s = -1, s' = 3, t = 2, t' = -5, q = 0, pc = 4.
> state 12: r = 1, r' = 0, s = -1, s' = 3, t = 2, t' = -5, q = 0, pc = 4.
> state 13: r = 

**Exemplo onde k passos não é suficiente para determinar gcd(a,b).**

In [6]:
#nº de Passos:
k=14

a=2300
b=310

#tamanho do bitvector:
n=20

gera_traco(k)

Input:
a = 2300 , b = 310 

Execução:
> state 0: r = 13, r' = 12, s = 11, s' = 10, t = 9, t' = 8, q = 6, pc = 0.
> state 1: r = 2300, r' = 310, s = 1, s' = 0, t = 0, t' = 1, q = 4, pc = 1.
> state 2: r = 2300, r' = 310, s = 1, s' = 0, t = 0, t' = 1, q = 14, pc = 2.
> state 3: r = 2300, r' = 310, s = 1, s' = 0, t = 0, t' = 1, q = 7, pc = 3.
> state 4: r = 310, r' = 130, s = 0, s' = 1, t = 1, t' = -7, q = 21, pc = 1.
> state 5: r = 310, r' = 130, s = 0, s' = 1, t = 1, t' = -7, q = 24, pc = 2.
> state 6: r = 310, r' = 130, s = 0, s' = 1, t = 1, t' = -7, q = 2, pc = 3.
> state 7: r = 130, r' = 50, s = 1, s' = -2, t = -7, t' = 15, q = 29, pc = 1.
> state 8: r = 130, r' = 50, s = 1, s' = -2, t = -7, t' = 15, q = 33, pc = 2.
> state 9: r = 130, r' = 50, s = 1, s' = -2, t = -7, t' = 15, q = 2, pc = 3.
> state 10: r = 50, r' = 30, s = -2, s' = 5, t = 15, t' = -37, q = 38, pc = 1.
> state 11: r = 50, r' = 30, s = -2, s' = 5, t = 15, t' = -37, q = 41, pc = 2.
> state 12: r = 50, r' = 30, s = -2, 

# **Provar que o programa nunca atinge o estado de erro.**

Para provar que o programa nunca atinge o estado de erro usaremos a função **bmc_always** que usamos durante as aulas, esta receberá a invariante/caso de erro juntamente com o numero de passos K.

(A função **initinv** existe pois foi necessária uma nova forma de inicialização para a invariante de Bézout ser válida no estado inicial, esta será mencionada mais para a frente)

In [8]:
def initinv(state):
    PC = Equals(state['pc'],BV(0,n)) 
    A = And(BVUGT(state['a'],BV(0,n)),Equals(state['a'], BV(a,n)))
    B = And(BVUGT(state['b'],BV(0,n)),Equals(state['b'], BV(b,n)))
    R = Equals(state['r'],BV(a,n))
    S = Equals(state['s'],BV(1,n))
    T = Equals(state['t'],BV(0,n))
    return And(PC,A,B,R,S,T)

def bmc_always(inv,K):
    # completar
    # K é o tamanho do traço
    # inv = invariante, ou estado de erro
        
    with Solver() as solver:
        states = [declare(i) for i in range(K+1)] # declarar estados
        solver.add_assertion(initinv(states[0])) # satisfazer o estado inicial

        for k in range(K):
            if k>0:
                solver.add_assertion(trans(states[k-1],states[k]))

            solver.push()
            solver.add_assertion(Not(inv(states[k]))) # ver se o invariante não satisfaz no estado k 
           # print(not(inv(states[k])))

            if solver.solve(): # inv nao satisfaz
                print(f">Invariant or nonerror case does not holds for {k+1} first states. \nCounter-example:")
                for i,s in enumerate(states[:k+1]):
                    print(f"> state {i}: r = {solver.get_value(s['r']).bv_signed_value()}, r' = {solver.get_value(s['r_']).bv_signed_value()}, s = {solver.get_value(s['s']).bv_signed_value()}, s' = {solver.get_value(s['s_']).bv_signed_value()}, t = {solver.get_value(s['t']).bv_signed_value()}, t' = {solver.get_value(s['t_']).bv_signed_value()}, q = {solver.get_value(s['q']).bv_signed_value()}, pc = {solver.get_value(s['pc']).bv_signed_value()}.")
                return
            else: #inv satisfaz (assertion not(inv(states[k]))) falhou porque inv satisfez
                if k==K-1:
                    print(f">Invariant or nonerror case holds for {K} first states.\n")

            solver.pop()

Criamos então a função **erros** que representa o inverso do estado de erro $r = 0$ usando o seguinte predicado:
$$(r>0 \vee r<0)$$
Se esta for então valida em todos os estados é porque $r = 0$ nunca acontece

In [9]:
def erros(state):
    return Or(BVUGT(state['r'],BV(0,n)),BVULT(state['r'],BV(0,n)))# r>1 || r<1  se r=0 dá verdade quando não é 0

bmc_always(erros,20)

>Invariant or nonerror case holds for 20 first states.



Para o outro caso de erro de nenhuma variável dar overflow criamos então a função **overfl** que representa nenhuma variável dar overflow.Usamos então o seguinte predicado tendo em conta que $2 ^ n -1$ é o maior numero num bitvetor de tamanho n:
$$(!(r>((2^n)-1) \vee r'>((2^n)-1)\vee s>((2^n)-1)\vee s'>((2^n)-1)\vee t>((2^n)-1)\vee t'>((2^n)-1)))$$
Se esta for válida em todos os estados é porque nenhuma destas varíavéis dá overflow ao longo do programa.

In [10]:
def overfl(state):
    valoroverfl=BV(pow(2,n)-1,n)
    R=Not(Or(BVUGT(state['r'],valoroverfl),#r overflow 
              BVUGT(state['r_'],valoroverfl),#r' overflow
              BVUGT(state['s'],valoroverfl),#s overflow
              BVUGT(state['s_'],valoroverfl),#s' overflow
              BVUGT(state['t'],valoroverfl),#t overflow
              BVUGT(state['t_'],valoroverfl)#t' overflow
                       ))# dá verdade quando não tivermos um overflow
    #r,s,t,r',s',t'>2 ^31 - 1
    #print(valoroverfl)
    #print(R)
    return R

bmc_always(overfl,20)

>Invariant or nonerror case holds for 20 first states.



# **Provar que a relação de Bézout$\,$ $\,a*s + b*t = r\,$é um invariante do algoritmo**

Para provar que a relação de Bézout é um invariante do algortimo, usamos então **bmc_always** usada nas aulas juntamente com **initinv** previamente explicado.

Por fim criamos então a função **invariante** que representará a relação de Bézout.Usamos então o seguinte predicado:
$$(r=((a\times s)+(b \times t))$$

Se esta for válida em todos os estados é porque representa uma invariante do programa.

In [11]:
def invariante(state):
    #a∗s+b∗t
    return Equals(state['r'],BVAdd(BVMul(BV(a,n),state['s']),BVMul(BV(b,n),state['t'])))

bmc_always(invariante,20)

>Invariant or nonerror case holds for 20 first states.



Além disso criamos uma invariante incorreta de forma a exemplificar o que ocorre caso seja dado uma invariante inválida, esta tem então o seguinte predicado:
$$(t = 0 \vee t = 1 )$$

Esta não é válida em todos os estados, o que não vai de encontro com a definição de invariante.

In [12]:
def invarianteteste(state):
    return Or(Equals(state['t'],BV(0,n)),Equals(state['t'],BV(1,n)))

bmc_always(invarianteteste,20)

>Invariant or nonerror case does not holds for 8 first states. 
Counter-example:
> state 0: r = 5, r' = 0, s = 1, s' = 0, t = 0, t' = 0, q = 0, pc = 0.
> state 1: r = 5, r' = 3, s = 1, s' = 0, t = 0, t' = 1, q = 0, pc = 1.
> state 2: r = 5, r' = 3, s = 1, s' = 0, t = 0, t' = 1, q = 0, pc = 2.
> state 3: r = 5, r' = 3, s = 1, s' = 0, t = 0, t' = 1, q = 1, pc = 3.
> state 4: r = 3, r' = 2, s = 0, s' = 1, t = 1, t' = -1, q = 0, pc = 1.
> state 5: r = 3, r' = 2, s = 0, s' = 1, t = 1, t' = -1, q = 0, pc = 2.
> state 6: r = 3, r' = 2, s = 0, s' = 1, t = 1, t' = -1, q = 1, pc = 3.
> state 7: r = 2, r' = 1, s = 1, s' = -1, t = -1, t' = 2, q = 0, pc = 1.
